In [59]:
from pymongo import MongoClient
import pandas as pd

In [60]:
uri = "mongodb://localhost:27017"
client = MongoClient(uri)

db = client["Books_Recommendation"]
collection = db["Blogs"]

In [61]:
data = collection.find()
blogs = pd.DataFrame(data)
blogs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           40 non-null     object
 1   bookTitle     40 non-null     object
 2   author        40 non-null     object
 3   department    40 non-null     object
 4   studentId     40 non-null     object
 5   introduction  40 non-null     object
 6   content       40 non-null     object
 7   conclusion    40 non-null     object
 8   date          17 non-null     object
 9   title         40 non-null     object
dtypes: object(10)
memory usage: 3.3+ KB


In [62]:
blogs['tags']=(
    blogs['title'].fillna('').astype(str).apply(lambda x: x.split()) +
    blogs['author'].fillna('').astype(str).apply(lambda x: x.split()) +
    blogs['bookTitle'].fillna('').astype(str).apply(lambda x: x.split()) +
    blogs['content'].fillna('').astype(str).apply(lambda x: x.split()) +
    blogs['introduction'].fillna('').astype(str).apply(lambda x: x.split()) +
    blogs['conclusion'].fillna('').astype(str).apply(lambda x: x.split()) +
    blogs['author'].fillna('').astype(str).apply(lambda x: x.split()) +
    blogs['department'].fillna('').astype(str).apply(lambda x: x.split()) 
)

In [63]:
blogs.head(2)

,_id,bookTitle,author,department,studentId,introduction,content,conclusion,date,title,tags
0,685aeb543504ff8cc1c4dbca,The Kite Runner,MD Muzahidul Islam,CSE,19CSE063,Khaled Hosseini’s The Kite Runner explores the...,"The Kite Runner follows the life of Amir, a pr...","Deeply emotional and beautifully written, The ...",2025-06-24T18:15:48.204Z,"The Kite Runner: A Story of Friendship, Loss, ...","[The, Kite, Runner:, A, Story, of, Friendship,..."
1,685aec1a3504ff8cc1c4dbcb,Animal Farm,Tushar,EEE,21EEE004,George Orwell’s Animal Farm is a brilliant pol...,Animal Farm begins with a group of farm animal...,Animal Farm is a timeless warning about the da...,2025-06-24T18:19:06.381Z,Animal Farm: A Powerful Allegory of Power and ...,"[Animal, Farm:, A, Powerful, Allegory, of, Pow..."


In [64]:
new_tags =blogs.drop(columns=['title', 'date', 'studentId', 'content', 'introduction', 'conclusion'])

In [65]:
new_tags['tags'] = new_tags['tags'].apply(lambda x: " ".join(x))
new_tags.head(2)

,_id,bookTitle,author,department,tags
0,685aeb543504ff8cc1c4dbca,The Kite Runner,MD Muzahidul Islam,CSE,"The Kite Runner: A Story of Friendship, Loss, ..."
1,685aec1a3504ff8cc1c4dbcb,Animal Farm,Tushar,EEE,Animal Farm: A Powerful Allegory of Power and ...


In [66]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [67]:
from sklearn.metrics.pairwise import cosine_similarity

In [68]:
new_tags = new_tags[new_tags['tags'].notnull() & (new_tags['tags'] != '')]

# Now vectorize
vector = cv.fit_transform(new_tags['tags']).toarray()

# Then compute similarity
similarity = cosine_similarity(vector)


In [69]:
vector.shape, similarity.shape

((40, 1813), (40, 40))

In [70]:
def recommend(blogs):
    index = new_tags[new_tags['bookTitle'] == blogs].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    for i in distances[1:6]:
        print(new_tags.iloc[i[0]]['bookTitle'])

In [71]:
recommend('The Kite Runner')

The Kite Runner
The Kite Runner
The Yearling
 The Price of Eternal Youth
Me and Earl and the Dying Girl


In [53]:
import pickle

In [54]:
pickle.dump(new_tags,open('blog_list.pkl','wb'))
pickle.dump(similarity,open('similarity_blog.pkl','wb'))